# Scikit-learn : model_selection

## 붓꽃 품종 예측하기

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [23]:
# 붓꽃 데이터 세트 불러오기
import pandas as pd

iris = load_iris()
iris_data = iris.data
iris_y = iris.target

iris_df = pd.DataFrame(data = iris_data, columns = iris.feature_names)
iris_df['label'] = iris_y
iris_df.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### 데이터 세트 분리 : train, test set 구분

In [29]:
X_train, X_test, y_train, y_test = train_test_split(iris_data, iris_y, test_size=0.2, random_state=11)

### 모델 학습 : 모델 객체화 및 train set 학습

In [30]:
dt_clf = DecisionTreeClassifier(random_state=11)
dt_clf.fit(X_train,y_train)

DecisionTreeClassifier(random_state=11)

### 예측 수행 : test set 예측

In [31]:
pred = dt_clf.predict(X_test)

### 평가

In [32]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))

0.9333333333333333
[[ 9  0  0]
 [ 0 10  0]
 [ 0  2  9]]


## 사이킷런의 기반 프레임워크 익히기

### Estimator 이해 및 fit(), predict() 매서드

In [33]:
# Classifier : 분류 알고리즘을 구현한 클래스
# Regressor : 회귀 알고리즘을 구현한 클래스
# Estimator = Classifier + Regressor
# 즉, 지도학습의 모든 알고리즘을 구현한 클래스를 Estimator라고 함.

### 사이킷런의 주요 모듈

#### 예제 데이터

* sklearn.datasets : 사이킷런에 내장되어 예제로 제공하는 데이터 세트

#### 피처 처리

* sklearn.preprocessing : 데이터 전처리에 필요한 다양한 가공 기능 제공(문자열을 숫자형 코드 값으로 인코딩, 정규화, 스케일링 등)

* sklearn.feature_selection : 알고리즘에 큰 영향을 미치는 피처를 우선순위대로 셀렉션 작업을 수행하는 다양한 기능 제공

* sklearn.feature_extraction : 텍스트 데이터나 이미지 데이터의 벡터화된 피처를 추출하는데 사용됨

#### 피처 처리 & 차원 축소

* sklearn.decomposition : 차원 축소와 관련한 알고리즘을 지원하는 모듈(PCA, NMF, SVD 등)

#### 데이터 분리, 검증 & 파라미터 튜닝

* sklearn.model_selection : 교차 검증을 위한 학습용/테스트용 분리, 그리고 그리드 서치로 최적 파라미터 추출

#### 평가

* sklearn.metrics : 분류, 회귀, 클러스터링, 페어와이즈에 대한 다양한 성능 측정 방법 제공(Accuracy_score, confusion_matrics 등)

#### ML 알고리즘

* sklearn.ensemble : 앙상블 알고리즘 제공(랜덤 포레스트, 에이다 부스트, 그래디언트 부스팅 등을 제공)

* sklearn.linear_model : 선형 회귀, 릿지, 라쏘 및 로지스틱 회귀 등 회귀 관련 알고리즘 지원, SGD 알고리즘 제공

* sklearn.naive_bayes : 나이브 베이즈 알고리즘 제공. 가우시안 NB, 다항 분포 NB 등

* sklearn.neighbors : 최근접 이웃 알고리즘 제공. K-NN

* sklearn.svm : 서포트 벡터 머신 알고리즘 제공

* sklearn.tree : 의사 결정 트리 알고리즘 제공

* sklearn.cluster : 비지도 클러스터링 알고리즘 제공.(K-평균, 계층형, DBSCAN 등)

#### 유틸리티

* sklearn.pipeline : 피처 처리 등의 변환과 ML 알고리즘 학습, 예측 등을 함께 묶어서 실행할 수 있는 유틸리티 제공

## 교차검증

* 데이터 편중을 막기 위해 별도의 여러 세트로 구성된 학습 데이터 세트와 검증 데이터 세트에서 학습과 평가를 수행
* 학습 데이터를 다시 분할하여 학습 데이터와 학습된 모델의 성능을 일차 평가하는 검증 데이터로 나눔.

### K 폴드 교차 검증
##### - k개의 데이터 폴드 세트를 만들어서 k번만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행하는 방법.

k = 5라고 하면,
1. 학습 데이터를 5등분
2. 1,2,3,4번으로 학습 5번으로 검증
3. 1,2,3,5번으로 학습 4번으로 검증 ... 3, 2, 1을 검증 데이터로 놓고 반복
4. 5개의 예측 평가를 구한 후 이를 평균해서 k 폴드 평가 결과로 반영.

In [43]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
data = iris.data
y = iris.target
dt_clf = DecisionTreeClassifier(random_state=11)

kfold = KFold(n_splits=5)
cv_accuracy = []

for train_index, test_index in kfold.split(data): # split()을 호출하면 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 변환
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    accuracy = accuracy_score(y_test, pred)
    cv_accuracy.append(accuracy)

np.mean(cv_accuracy)

0.9199999999999999

### Stratified K 폴드
##### - 불균형한 분포도를 가진 레이블 데이터 집합을 위한  K 폴드 방식  ( y의 특정 레이블 값이 특이하게 많거나 매우 적은 경우 )

ex) 대출 사기 데이터
* 총 데이터 세트 1억 건, 대출 사기 1000건으로 약 0.0001%의 비율.
* 만약 K 폴드 방식을 사용하는 경우, 어느 데이터 세트에는 대출 사기 건수가 많이 포함하고 어느 데이터 세트에는 대출 사기가 존재하지 않을 수도 있다.

In [48]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
skf_accuracy = []

# StratifiedKFold는 레이블 데이터 분포도에 따라 학습/검증 데이터를 나누기 때문에 split() 인자로 레이블 데이터 세트가 반드시 필요.
for train_index, test_index in skf.split(data, y):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred_skf = dt_clf.predict(X_test)
    
    skf_accuracy.append(accuracy_score(pred_skf, y_test))

print(np.mean(skf_accuracy))

0.96


* 일반적으로 분류에서의 교차 검증은 Stratified K 폴드, 회귀에서의 교차 검증은 K 폴드를 사용. ( 회귀의 결정값은 연속된 숫자값이기 때문에 )

### cross_val_score()
: cross_val_score(estimator, X, y=None, scoring=None, cv=None)
* estimator : Classifier or Regressor를 객체화한 인스턴스 ( classifier가 입력되면 Stratified K fold, Regressor가 입력되면 K fold 방식으로 분할 )
* X : 피처 데이터 세트
* y : 레이블 데이터 세트
* scoring : 예측 성능 평가 지표
* cv : 교차 검증 폴드 수

In [55]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dt_clf, data, y, scoring = 'accuracy', cv = 3)

print(scores)
print(np.mean(scores))

[0.98 0.92 0.98]
0.96


## GridSearchCV - 교차 검증과 최적 하이퍼 파라미터 튜닝을 한 번에
: GridSearchCV(estimator, param_grid = , scoring = , cv = , refit = )
* estimator : Classifier, Regressor, Pipeline 사용 가능
* param_grid : key + 리스트 값을 가지는 딕셔너리.
* scoring : 예측 성능을 측정할 평가 방법 지정.
* cv : 교차 검증을 위해 분할되는 학습/데이터 세트의 개수 지정.
* refit : 디폴트가 True이며 True 설정 시 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습.

In [73]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth' : [1,2,3], 'min_samples_split' : [2,3]}

# 3번의 교차 검증을 통해 parameters 중에 가장 최적의 파라미터를 찾아 학습.(파라미터에 대한 학습)
grid_dt_clf = GridSearchCV(dt_clf, param_grid=parameters, cv = 3, refit = True)

# train set을 이용하여 학습(데이터에 대한 학습)
grid_dt_clf.fit(X_train, y_train)

# GridSearchCV 결과를 DataFrame으로 변환
result_df = pd.DataFrame(grid_dt_clf.cv_results_)
# print(result_df)

# 최적 파라미터 및 정확도
print(grid_dt_clf.best_params_)
print(grid_dt_clf.best_score_)

# GridSearchCV의 refit으로 학습된 최적의 estimator 반환
estimator = grid_dt_clf.best_estimator_

# 최적의 estimator과 test set를 활용한 평가
pred_grid = estimator.predict(X_test)
print(accuracy_score(y_test, pred_grid))

{'max_depth': 3, 'min_samples_split': 2}
0.9598930481283423
0.98
